In [8]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Input, add, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import concatenate

In [6]:

def naive_resnet_block(in_layer, f):
    out_layer = Conv2D(f, (3,3), padding = 'same', activation = 'relu')(in_layer)
    out_layer = Conv2D(f, (3,3), padding = 'same', activation = 'linear')(out_layer)

    ## my first intuition was to concat the out with the input layer. But in resnet, we are adding the skip connection. This means 
    ## that the number of filters in input layer should be the same as the last conv layer
    
    ## notice: we add first and then apply activation to the addition. Hence last conv layer has linear activation.
    out_layer = add([out_layer, in_layer])
    
    out_layer = Activation('relu')(out_layer)
    return out_layer


def residual_block(in_layer, f):
    
    out_layer = Conv2D(f, (3,3), padding = 'same', activation = 'relu')(in_layer)
    out_layer = Conv2D(f, (3,3), padding = 'same', activation = 'linear')(out_layer)
    
    ## we add a 1*1 convolution stage (referred as projection layer)
    
    if in_layer.shape[-1]!=f:
        in_layer = Conv2D(f, (1,1), padding = 'same', activation = 'relu')(in_layer)
     
    out_layer = add([out_layer, in_layer])
    
    out_layer = Activation('relu')(out_layer)
    
    return out_layer

In [11]:
visible = Input(shape=(256, 256, 3))
# add inception block 1
layer = residual_block(visible, 64)
# add inception block 1
layer = residual_block(layer, 32)
# create model
model = Model(inputs=visible, outputs=layer)
# summarize model
model.summary()
# plot model architecture
plot_model(model, show_shapes=True, to_file='module.png')

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d_11 (Conv2D)              (None, 256, 256, 64) 1792        input_6[0][0]                    
__________________________________________________________________________________________________
conv2d_12 (Conv2D)              (None, 256, 256, 64) 36928       conv2d_11[0][0]                  
__________________________________________________________________________________________________
conv2d_13 (Conv2D)              (None, 256, 256, 64) 256         input_6[0][0]                    
______________________________________________________________________________________________

![](/helper/inception.JPG)

![](helper/resnet.jpg)

## Learnings 

* my first intuition was to concat the out with the input layer. But in resnet, we are adding the skip connection. This means 
  that the number of filters in input layer should be the same as the last conv layer
    
* notice: we add first and then apply activation to the addition. Hence last conv layer has linear activation.
    
 * we add a 1*1 convolution stage (referred as projection layer) to the input layer to make no of filters equal to last conv layer. 